In [ ]:
import torch
import random
import numpy as np


SEED = 1000

def setup_reproducibility():
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(False, warn_only=True)
    torch.set_float32_matmul_precision("high")

setup_reproducibility()

In [ ]:
import os

path = "/kaggle/input/dig-4-bio-raman-transfer-learning-challenge"
files = os.listdir(path)
[(i, files[i]) for i in range(len(files))]

In [ ]:
import pandas as pd

df = pd.read_csv(os.path.join(path, files[5]))
df

In [ ]:
input_cols = df.columns[1:2049]
target_cols = df.columns[2050:]

In [ ]:
targets  = df[target_cols].dropna().to_numpy()
targets[:5]

In [ ]:
df = df[input_cols]
df['Unnamed: 1'] = df['Unnamed: 1'].str.replace('[\[\]]', '', regex=True).astype('int64')
df['Unnamed: 2048'] = df['Unnamed: 2048'].str.replace('[\[\]]', '', regex=True).astype('int64')

In [ ]:
import numpy as np

inputs = []
for i in range(0, len(df), 2):
    row1 = df.iloc[i].to_numpy()
    row2 = df.iloc[i+1].to_numpy()
    i = np.concatenate([row1, row2])
    inputs.append(i)
    
inputs = np.stack(inputs)
inputs.shape

In [ ]:
from sklearn.model_selection import train_test_split


train_inputs, eval_inputs, train_targets, eval_targets = train_test_split(
        inputs,
        targets,                      
        test_size=0.2,
        random_state=1000,
        shuffle=True
)

train_inputs.shape, eval_inputs.shape, train_targets.shape, eval_targets.shape

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_inputs = scaler.fit_transform(train_inputs)
eval_inputs = scaler.transform(eval_inputs)

In [ ]:
train_inputs = torch.tensor(train_inputs).float()
train_targets = torch.tensor(train_targets).float()
eval_inputs = torch.tensor(eval_inputs).float()
eval_targets = torch.tensor(eval_targets).float()

In [ ]:
from torch.utils.data import TensorDataset

train_ds = TensorDataset(train_inputs, train_targets)
eval_ds = TensorDataset(eval_inputs, eval_targets)

In [ ]:
from torch.utils.data import DataLoader


def build_loader(
    SEED,
    ds,
    train=True,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    drop_last=True,
    pin_memory=True,
    persistent_workers=False,
):
    def seed_worker(worker_id):
        worker_seed = torch.initial_seed() % 2**32
        np.random.seed(worker_seed)
        random.seed(worker_seed)

    generator = torch.Generator()
    generator.manual_seed(SEED if train else SEED+1)

    return DataLoader(
        ds,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        pin_memory=pin_memory,
        drop_last=drop_last,
        persistent_workers=persistent_workers,
        worker_init_fn=seed_worker,
        generator=generator,
        #sampler=DistributedSampler(
        #    train_ds,
        #    shuffle=True,
        #    drop_last=True,
        #    seed=config.seed
        #)
    )
    
    
def return_dls(train_ds, eval_ds):
    train_dl = build_loader(
        SEED,
        train_ds,
        train=True,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=0,
        drop_last=False,
        pin_memory=True,
        persistent_workers=False,
    )

    eval_dl = build_loader(
        SEED,
        eval_ds,
        train=False,
        batch_size=20,
        shuffle=False,
        num_workers=0,
        drop_last=False,
        pin_memory=True,
        persistent_workers=False,
    )
    
    return train_dl, eval_dl

In [ ]:
import torch.nn.functional as F
from sklearn.metrics import r2_score


def loss_fn(logits, targets):
    logits = logits.view(-1)
    targets = targets.view(-1)
    return F.mse_loss(logits, targets)


def metric_fn(logits, targets):
    preds = logits.cpu().detach().numpy()
    targets = targets.cpu().detach().numpy()
    one = r2_score(targets[:, 0], preds[:, 0])
    two = r2_score(targets[:, 1], preds[:, 1])
    three = r2_score(targets[:, 2], preds[:, 2])
    mean_r2 = (one + two + three) / 3
    return one, two, three, mean_r2

In [ ]:
import torch.nn as nn

class Model(nn.Module):
    """
    MLP optimized for Raman spectroscopy concentration prediction
    """
    def __init__(self, input_size=4096, hidden_sizes=[128], 
                 output_size=3, dropout_rate=0.3, use_batch_norm=True):
        super().__init__()
        
        self.input_size = input_size
        self.output_size = output_size
        self.use_batch_norm = use_batch_norm
        
        # Build the network layers
        layers = []
        prev_size = input_size
        
        for i, hidden_size in enumerate(hidden_sizes):
            # Linear layer
            layers.append(nn.Linear(prev_size, hidden_size))
            
            # Batch normalization
            if use_batch_norm:
                layers.append(nn.BatchNorm1d(hidden_size))
            
            # Activation
            layers.append(nn.GELU())
            
            # Dropout (not on the last hidden layer)
            if i < len(hidden_sizes) - 1:
                layers.append(nn.Dropout(dropout_rate))
            
            prev_size = hidden_size
        
        # Output layer
        layers.append(nn.Linear(prev_size, output_size))
        
        self.network = nn.Sequential(*layers)
        
        # Initialize weights
        self._initialize_weights()
        self.d = nn.Dropout(dropout_rate)
    
    def _initialize_weights(self):
        """Initialize network weights using Xavier/Glorot initialization"""
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_normal_(module.weight)
                if module.bias is not None:
                    nn.init.constant_(module.bias, 0)
    
    def forward(self, x):
        x = self.d(x)
        return self.network(x)


In [ ]:
import warnings
from transformers import get_cosine_schedule_with_warmup
from tqdm.auto import tqdm

#warnings.filterwarnings("ignore")

MODEL_NAME = "MLP.Baseline.AdamW"
EPOCHS = 4000
BATCH_SIZE = 16
WD = 1e-4
LR = 1e-4
DROPOUT = 0.3
SCORE = float('-inf')
LOG = False
RESUME = False
device = "cuda" if torch.cuda.is_available() else "cpu"



model = Model(dropout_rate=DROPOUT).to(device)
#model = nn.DataParallel(model)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD, foreach=True)
scaler = torch.GradScaler(device)
train_dl, eval_dl = return_dls(train_ds, eval_ds)

total_training_steps = len(train_dl) * EPOCHS
warmup_steps = int(total_training_steps * 0.05)  # e.g. 5% warmup
scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_training_steps
)

if LOG:
    neptune_run = setup_neptune()



for epoch in tqdm(range(EPOCHS)):
    model.train()
    total_loss = 0.0
    all_logits = []
    all_targets = []
    
    for inputs, targets in train_dl:
        inputs = inputs.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)
        
        noise = torch.randn_like(inputs) * 0.1  # std=0.1
        inputs = inputs + noise

        with torch.autocast(device_type=device, dtype=torch.float16, cache_enabled=True):
            logits = model(inputs)
            loss = loss_fn(logits, targets)
            
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
        scheduler.step()
        
        total_loss += loss.detach().cpu()
        all_logits.append(logits.detach().cpu())
        all_targets.append(targets.detach().cpu())
    
    all_logits = torch.cat(all_logits)
    all_targets = torch.cat(all_targets)

    one, two, three, r2 = metric_fn(all_logits, all_targets)
    total_loss = total_loss / len(train_dl)
    
    
    model.eval()
    eval_total_loss = 0.0
    eval_all_logits = []
    eval_all_targets = []

    for inputs, targets in eval_dl:
        inputs = inputs.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)

        with torch.inference_mode():
            with torch.autocast(device_type=device, dtype=torch.float16, cache_enabled=True):
                logits = model(inputs)
                loss = loss_fn(logits, targets)

        eval_total_loss += loss.detach().cpu()
        eval_all_logits.append(logits.detach().cpu())
        eval_all_targets.append(targets.detach().cpu())
    
    eval_all_logits = torch.cat(eval_all_logits)
    eval_all_targets = torch.cat(eval_all_targets)
   
    eval_one, eval_two, eval_three, eval_r2 = metric_fn(eval_all_logits, eval_all_targets)
    eval_total_loss = eval_total_loss / len(eval_dl)
    
    if r2 > SCORE:
        SCORE = r2
        data = {"state_dict": model.state_dict()}
        data["epoch"] = epoch 
        data["score"] = SCORE
        torch.save(data, "/kaggle/working/ckpt.pt")
    
    if LOG:
        neptune_run["train/loss"].append(LOSS)
        neptune_run["eval/loss"].append(EVAL_LOSS)
        neptune_run["train/auroc"].append(auroc)
        neptune_run["eval/auroc"].append(eval_auroc)
        
    if True:
        print(
            f"Epoch: {epoch}, "
            f"train/loss: {total_loss:.4f}, "
            f"eval/loss: {eval_total_loss:.4f}, "
            f"train/r2: {r2:.4f}, "
            f"eval/r2: {eval_r2:.4f}, "
            f"train/one: {one:.4f}, "
            f"train/two: {two:.4f}, "
            f"train/three: {three:.4f}, "
            f"eval/one: {eval_one:.4f}, "
            f"eval/two: {eval_two:.4f}, "
            f"eval/three: {eval_three:.4f}, "
        )

In [ ]:
#def load_test():    train/r2: 0.8097    eval/r2: -0.7274, t
test = pd.read_csv(os.path.join(path, files[6]))

row1 = test.columns[1:].to_numpy().copy()
row1[-1] = "5611"
row1 = row1.astype(np.float64)


cols = test.columns[1:]
test = test[cols]
test[" 5611]"] = test[" 5611]"].str.replace('[\[\]]', '', regex=True).astype('int64')
test = test.to_numpy()

test = np.insert(test, 0, row1, axis=0)
test = test.reshape(-1, 2, 2048).reshape(96, 2*2048)
test = test.astype(np.float32)
test.shape, test.dtype

In [ ]:
#def scale():
test = scaler.transform(test)
preds = xgb.predict(test)
preds.shape

In [ ]:
#def prepare_test():
column_names = ['Glucose', 'Sodium Acetate', 'Magnesium Sulfate']
preds_df = pd.DataFrame(preds, columns=column_names)
preds_df.insert(0, 'ID', [i+1 for i in range(len(preds_df))])
preds_df

In [ ]:
#def save_test():
preds_df.to_csv("baseline.csv", index=False)
f = pd.read_csv("/kaggle/working/xgb_baseline.csv")
f